<a href="https://colab.research.google.com/github/kurek0010/neural-network-course/blob/master/05_regression/01_housing_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* @author: krakowiakpawel9@gmail.com  
* @site: e-smartdata.org

### Problem regresji - prognoza ceny mieszkań

Celem tego notebook'a jest zapoznanie się z problemami regresji. W odróżnieniu od problemów klasyfikacji, gdzie staramy się przyporządkować daną klasę, w modelach regresji przewidujemy ciągłą zmienną docelową.

### Spis treści:
1. [Import bibliotek](#a0)
2. [Załadowanie danych i wstępna eksploracja](#a1)
3. [Podział na zbiór treningowy oraz testowy](#a2)
4. [Standaryzacja danych](#a3)
5. [Budowa modelu](#a4)
6. [Trenowanie sieci](#a5)





### <a name='a0'></a> 1. Import biblotek

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

np.set_printoptions(precision=12, suppress=True, linewidth=150)
pd.options.display.float_format = '{:.6f}'.format
tf.__version__

'2.18.0'

### <a name='a1'></a> 2. Załadowanie danych i wstępna eksploracja

In [2]:
raw_dataset = pd.read_csv('https://storage.googleapis.com/esmartdata-courses-files/ann-course/housing.csv')
raw_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [3]:
dataset = raw_dataset.copy()
dataset.head()

longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0 -122.230000 37.880000           41.000000   880.000000      129.000000   
1 -122.220000 37.860000           21.000000  7099.000000     1106.000000   
2 -122.240000 37.850000           52.000000  1467.000000      190.000000   
3 -122.250000 37.850000           52.000000  1274.000000      235.000000   
4 -122.250000 37.850000           52.000000  1627.000000      280.000000   

   population  households  median_income  median_house_value ocean_proximity  
0  322.000000  126.000000       8.325200       452600.000000        NEAR BAY  
1 2401.000000 1138.000000       8.301400       358500.000000        NEAR BAY  
2  496.000000  177.000000       7.257400       352100.000000        NEAR BAY  
3  558.000000  219.000000       5.643100       341300.000000        NEAR BAY  
4  565.000000  259.000000       3.846200       342200.000000        NEAR BAY

In [4]:
dataset.tail()

longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
20635 -121.090000 39.480000           25.000000  1665.000000      374.000000   
20636 -121.210000 39.490000           18.000000   697.000000      150.000000   
20637 -121.220000 39.430000           17.000000  2254.000000      485.000000   
20638 -121.320000 39.430000           18.000000  1860.000000      409.000000   
20639 -121.240000 39.370000           16.000000  2785.000000      616.000000   

       population  households  median_income  median_house_value  \
20635  845.000000  330.000000       1.560300        78100.000000   
20636  356.000000  114.000000       2.556800        77100.000000   
20637 1007.000000  433.000000       1.700000        92300.000000   
20638  741.000000  349.000000       1.867200        84700.000000   
20639 1387.000000  530.000000       2.388600        89400.000000   

      ocean_proximity  
20635          INLAND  
20636          INLAND  
20637          INLAND  
20638          INLAND  
20639          INLAND

In [5]:
dataset.isnull().sum() / len(dataset)

longitude            0.000000
latitude             0.000000
housing_median_age   0.000000
total_rooms          0.000000
total_bedrooms       0.010029
population           0.000000
households           0.000000
median_income        0.000000
median_house_value   0.000000
ocean_proximity      0.000000
dtype: float64

In [6]:
dataset.dropna(inplace=True)

dataset.isnull().sum() / len(dataset)

longitude            0.000000
latitude             0.000000
housing_median_age   0.000000
total_rooms          0.000000
total_bedrooms       0.000000
population           0.000000
households           0.000000
median_income        0.000000
median_house_value   0.000000
ocean_proximity      0.000000
dtype: float64

In [7]:
dataset.describe()

longitude     latitude  housing_median_age  total_rooms  \
count 20433.000000 20433.000000        20433.000000 20433.000000   
mean   -119.570689    35.633221           28.633094  2636.504233   
std       2.003578     2.136348           12.591805  2185.269567   
min    -124.350000    32.540000            1.000000     2.000000   
25%    -121.800000    33.930000           18.000000  1450.000000   
50%    -118.490000    34.260000           29.000000  2127.000000   
75%    -118.010000    37.720000           37.000000  3143.000000   
max    -114.310000    41.950000           52.000000 39320.000000   

       total_bedrooms   population   households  median_income  \
count    20433.000000 20433.000000 20433.000000   20433.000000   
mean       537.870553  1424.946949   499.433465       3.871162   
std        421.385070  1133.208490   382.299226       1.899291   
min          1.000000     3.000000     1.000000       0.499900   
25%        296.000000   787.000000   280.000000       2.563700   
50%        435.000000  1166.000000   409.000000       3.536500   
75%        647.000000  1722.000000   604.000000       4.744000   
max       6445.000000 35682.000000  6082.000000      15.000100   

       median_house_value  
count        20433.000000  
mean        206864.413155  
std         115435.667099  
min          14999.000000  
25%         119500.000000  
50%         179700.000000  
75%         264700.000000  
max         500001.000000

In [8]:
dataset.describe(include=['object'])

ocean_proximity
count            20433
unique               5
top          <1H OCEAN
freq              9034

In [9]:
dataset.ocean_proximity.value_counts()

ocean_proximity
<1H OCEAN     9034
INLAND        6496
NEAR OCEAN    2628
NEAR BAY      2270
ISLAND           5
Name: count, dtype: int64

In [10]:
px.histogram(dataset, x='median_house_value')

In [11]:
dataset.median_house_value.value_counts()

median_house_value
500001.000000    958
137500.000000    119
162500.000000    116
112500.000000    103
187500.000000     92
                ... 
397400.000000      1
492000.000000      1
476300.000000      1
378600.000000      1
398500.000000      1
Name: count, Length: 3833, dtype: int64

In [12]:
index_to_drop = dataset[dataset.median_house_value == 500001].index
dataset = dataset.drop(index=index_to_drop)
px.histogram(dataset, x='median_house_value')

In [13]:
dataset_dummies = pd.get_dummies(dataset, drop_first=True)
dataset_dummies.head()

longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0 -122.230000 37.880000           41.000000   880.000000      129.000000   
1 -122.220000 37.860000           21.000000  7099.000000     1106.000000   
2 -122.240000 37.850000           52.000000  1467.000000      190.000000   
3 -122.250000 37.850000           52.000000  1274.000000      235.000000   
4 -122.250000 37.850000           52.000000  1627.000000      280.000000   

   population  households  median_income  median_house_value  \
0  322.000000  126.000000       8.325200       452600.000000   
1 2401.000000 1138.000000       8.301400       358500.000000   
2  496.000000  177.000000       7.257400       352100.000000   
3  558.000000  219.000000       5.643100       341300.000000   
4  565.000000  259.000000       3.846200       342200.000000   

   ocean_proximity_INLAND  ocean_proximity_ISLAND  ocean_proximity_NEAR BAY  \
0                   False                   False                      True   
1                   False                   False                      True   
2                   False                   False                      True   
3                   False                   False                      True   
4                   False                   False                      True   

   ocean_proximity_NEAR OCEAN  
0                       False  
1                       False  
2                       False  
3                       False  
4                       False

### <a name='a2'></a> 3. Podział na zbiór treningowy oraz testowy

In [14]:
train_dataset = dataset_dummies.sample(frac=0.8, random_state=0)
test_dataset = dataset_dummies.drop(train_dataset.index)

print(f'train_dataset length: {len(train_dataset)}')
print(f'test_dataset length: {len(test_dataset)}')

train_dataset length: 15580
test_dataset length: 3895


In [15]:
train_dataset.head()

longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
19234 -122.690000 38.510000           18.000000  3364.000000      501.000000   
1859  -124.140000 41.950000           21.000000  2696.000000      578.000000   
51    -122.270000 37.820000           43.000000  1868.000000      456.000000   
11192 -117.930000 33.820000           28.000000  2444.000000      555.000000   
20355 -118.960000 34.190000           16.000000  1807.000000      346.000000   

       population  households  median_income  median_house_value  \
19234 1442.000000  506.000000       6.685400       313000.000000   
1859  1208.000000  494.000000       2.275000       122400.000000   
51    1061.000000  407.000000       1.504500        93800.000000   
11192 1848.000000  567.000000       3.017900       198800.000000   
20355  587.000000  296.000000       1.981100       162500.000000   

       ocean_proximity_INLAND  ocean_proximity_ISLAND  \
19234                   False                   False   
1859                    False                   False   
51                      False                   False   
11192                   False                   False   
20355                   False                   False   

       ocean_proximity_NEAR BAY  ocean_proximity_NEAR OCEAN  
19234                     False                       False  
1859                      False                        True  
51                         True                       False  
11192                     False                       False  
20355                     False                       False

In [16]:
test_dataset.head()

longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
10 -122.260000 37.850000           52.000000  2202.000000      434.000000   
13 -122.260000 37.840000           52.000000   696.000000      191.000000   
19 -122.270000 37.840000           52.000000  1503.000000      298.000000   
26 -122.280000 37.850000           49.000000  1130.000000      244.000000   
40 -122.260000 37.830000           52.000000  1665.000000      419.000000   

    population  households  median_income  median_house_value  \
10  910.000000  402.000000       3.203100       281500.000000   
13  345.000000  174.000000       2.673600       191300.000000   
19  690.000000  275.000000       2.603300       162900.000000   
26  607.000000  239.000000       2.459700        93800.000000   
40  946.000000  395.000000       2.097800       155400.000000   

    ocean_proximity_INLAND  ocean_proximity_ISLAND  ocean_proximity_NEAR BAY  \
10                   False                   False                      True   
13                   False                   False                      True   
19                   False                   False                      True   
26                   False                   False                      True   
40                   False                   False                      True   

    ocean_proximity_NEAR OCEAN  
10                       False  
13                       False  
19                       False  
26                       False  
40                       False

In [17]:
px.scatter_matrix(train_dataset, dimensions=['median_house_value', 'housing_median_age', 'median_income', 'total_rooms'], color='median_house_value', height=700)

In [18]:
train_stats = train_dataset.describe()
train_stats.pop('median_house_value')
train_stats = train_stats.transpose()
train_stats

count        mean         std         min  \
longitude          15580.000000 -119.558780    2.006237 -124.350000   
latitude           15580.000000   35.648614    2.147016   32.550000   
housing_median_age 15580.000000   28.489217   12.505895    1.000000   
total_rooms        15580.000000 2620.014506 2195.654212    2.000000   
total_bedrooms     15580.000000  539.900578  424.499548    2.000000   
population         15580.000000 1441.193068 1160.542775    3.000000   
households         15580.000000  501.099487  385.039835    2.000000   
median_income      15580.000000    3.672427    1.570297    0.499900   

                           25%         50%         75%          max  
longitude          -121.750000 -118.490000 -117.990000  -114.470000  
latitude             33.930000   34.270000   37.730000    41.950000  
housing_median_age   18.000000   29.000000   37.000000    52.000000  
total_rooms        1441.000000 2112.000000 3119.000000 39320.000000  
total_bedrooms      299.000000  436.000000  647.000000  6445.000000  
population          801.000000 1179.000000 1746.000000 35682.000000  
households          283.000000  411.000000  605.000000  6082.000000  
median_income         2.519975    3.440500    4.581425    15.000100

In [19]:
train_labels = train_dataset.pop('median_house_value')
test_labels = test_dataset.pop('median_house_value')

### <a name='a3'></a> 4. Standaryzacja danych

In [20]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

In [21]:
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [22]:
normed_train_data.isnull().sum()

households                        0
housing_median_age                0
latitude                          0
longitude                         0
median_income                     0
ocean_proximity_INLAND        15580
ocean_proximity_ISLAND        15580
ocean_proximity_NEAR BAY      15580
ocean_proximity_NEAR OCEAN    15580
population                        0
total_bedrooms                    0
total_rooms                       0
dtype: int64

In [23]:
normed_train_data.head(3)

households  housing_median_age  latitude  longitude  median_income  \
19234    0.012727           -0.838742  1.332727  -1.560743       1.918729   
1859    -0.018438           -0.598855  2.934950  -2.283490      -0.889913   
51      -0.244389            1.160315  1.011351  -1.351396      -1.380585   

       ocean_proximity_INLAND  ocean_proximity_ISLAND  \
19234                     NaN                     NaN   
1859                      NaN                     NaN   
51                        NaN                     NaN   

       ocean_proximity_NEAR BAY  ocean_proximity_NEAR OCEAN  population  \
19234                       NaN                         NaN    0.000695   
1859                        NaN                         NaN   -0.200934   
51                          NaN                         NaN   -0.327599   

       total_bedrooms  total_rooms  
19234       -0.091639     0.338845  
1859         0.089751     0.034607  
51          -0.197646    -0.342501

In [24]:
normed_test_data.head(3)

households  housing_median_age  latitude  longitude  median_income  \
10   -0.257375            1.879976  1.025324  -1.346412      -0.298878   
13   -0.849521            1.879976  1.020666  -1.346412      -0.636075   
19   -0.587211            1.879976  1.020666  -1.351396      -0.680844   

    ocean_proximity_INLAND  ocean_proximity_ISLAND  ocean_proximity_NEAR BAY  \
10                     NaN                     NaN                       NaN   
13                     NaN                     NaN                       NaN   
19                     NaN                     NaN                       NaN   

    ocean_proximity_NEAR OCEAN  population  total_bedrooms  total_rooms  
10                         NaN   -0.457711       -0.249472    -0.190383  
13                         NaN   -0.944552       -0.821910    -0.876283  
19                         NaN   -0.647277       -0.569849    -0.508739

In [25]:
normed_test_data = normed_test_data.values
normed_train_data = normed_train_data.values

### <a name='a4'></a> 5. Budowa modelu

In [26]:
def build_model():
    model = Sequential()
    model.add(Dense(1024, kernel_regularizer='l2', activation='relu', input_shape=[len(train_dataset.keys())]))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1))

    model.compile(optimizer='adam',
                  loss='mse',
                  metrics=['mae', 'mse'])
    return model

In [27]:
model = build_model()
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 1024)                │          13,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 603,905 (2.30 MB)

 Trainable params: 603,905 (2.30 MB)

 Non-trainable params: 0 (0.00 B)

### <a name='a5'></a> 6. Trenowanie sieci

In [28]:
history = model.fit(normed_train_data, train_labels.values, epochs=150, validation_split=0.2, verbose=1, batch_size=32)

Epoch 1/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: nan - mae: nan - mse: nan - val_loss: nan - val_mae: nan - val_mse: nan
Epoch 2/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: nan - mae: nan - mse: nan - val_loss: nan - val_mae: nan - val_mse: nan
Epoch 3/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: nan - mae: nan - mse: nan - val_loss: nan - val_mae: nan - val_mse: nan
Epoch 4/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: nan - mae: nan - mse: nan - val_loss: nan - val_mae: nan - val_mse: nan
Epoch 5/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: nan - mae: nan - mse: nan - val_loss: nan - val_mae: nan - val_mse: nan
Epoch 6/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: nan - mae: nan - mse: nan - val_loss: nan - val_mae: nan - val_mse: nan
Epoch 7/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: nan - mae: nan - mse: nan - val_loss: nan - val_mae: nan - val_mse: nan
Epoch 8/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: nan - ma

In [33]:
def plot_hist(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    hist['rmse'] = np.sqrt(hist['mse'])
    hist['val_rmse'] = np.sqrt(hist['val_mse'])

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['mae'], name='mae', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_mae'], name='val_mae', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='MAE vs. VAL_MAE', xaxis_title='Epoki', yaxis_title='Mean Absolute Error', yaxis_type='log')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['rmse'], name='rmse', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_rmse'], name='val_rmse', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='RMSE vs. VAL_RMSE', xaxis_title='Epoki', yaxis_title='Root Mean Squared Error', yaxis_type='log')
    fig.show()

plot_hist(history)

In [30]:
for name, value in zip(model.metrics_names, model.evaluate(normed_test_data, test_labels.values)):
    print(f'{name:8}{value:.4f}')

122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: nan - mae: nan - mse: nan
loss    nan
compile_metricsnan


In [31]:
test_predictions = model.predict(normed_test_data).flatten()
test_predictions

122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


array([nan, nan, nan, ..., nan, nan, nan], dtype=float32)

In [32]:
pred = pd.DataFrame(test_labels)
pred['predictions'] = test_predictions
pred.head()

median_house_value  predictions
10       281500.000000          NaN
13       191300.000000          NaN
19       162900.000000          NaN
26        93800.000000          NaN
40       155400.000000          NaN

In [34]:
fig = px.scatter(pred, 'median_house_value', 'predictions')
fig.add_trace(go.Scatter(x=[0, 500000], y=[0, 500000], mode='lines'))
fig.show()

In [ ]:
pred.head()

median_house_value   predictions
10       281500.000000 206502.421875
13       191300.000000 173946.156250
19       162900.000000 147049.640625
26        93800.000000 124461.218750
40       155400.000000 145752.640625

In [ ]:
pred['error'] = pred['median_house_value'] - pred['predictions']
pred.head()

median_house_value   predictions         error
10       281500.000000 206502.421875  74997.578125
13       191300.000000 173946.156250  17353.843750
19       162900.000000 147049.640625  15850.359375
26        93800.000000 124461.218750 -30661.218750
40       155400.000000 145752.640625   9647.359375

In [ ]:
px.histogram(pred, 'error', marginal='rug', width=1000)